In [1]:
project_id = 'movie-genre-analysis-le-wagon'

In [2]:
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

Authenticated


In [33]:
from google.cloud import bigquery

client = bigquery.Client(project=project_id)

df = client.query('''
SELECT
  bx.*,
  tmdb.budget,
  tmdb.genre_1,
  tmdb.genre_2,
  tmdb.genre_3
FROM `movie-genre-analysis-le-wagon.Secondary_Data.box_office_cleaned` AS bx
INNER JOIN `movie-genre-analysis-le-wagon.Secondary_Data.tmdb_model_data_table` AS tmdb
ON LOWER(REGEXP_REPLACE(bx.movie_title, r'[^a-zA-Z0-9]', '')) = LOWER(REGEXP_REPLACE(tmdb.movie_title, r'[^a-zA-Z0-9]', ''))
WHERE bx.release_week_number = 4
ORDER BY bx.date
''').to_dataframe()
df

,unique_id,movie_title,weekly_rank,week_gross,current_total_gross,release_week_number,distributor,date,budget,genre_1,genre_2,genre_3
0,Alien_1979_W24,Alien,1,3783140,16466038,4,Twentieth Century Fox,1979-06-17 00:00:00+00:00,50000000,Science Fiction,Action,Horror
1,Alien_1979_W24,Alien,1,3783140,16466038,4,Twentieth Century Fox,1979-06-17 00:00:00+00:00,11000000,Horror,Science Fiction,None
2,Kramer vs. Kramer_1980_W1,Kramer vs. Kramer,1,7614000,24136206,4,Columbia Pictures,1980-01-13 00:00:00+00:00,8000000,Drama,None,None
3,Raging Bull_1980_W49,Raging Bull,2,141951,756917,4,Metro-Goldwyn-Mayer (MGM),1980-12-14 00:00:00+00:00,18000000,Drama,None,None
4,On Golden Pond_1981_W52,On Golden Pond,2,189014,564824,4,Universal Pictures,1982-01-03 00:00:00+00:00,15000000,Drama,Romance,None
...,...,...,...,...,...,...,...,...,...,...,...,...
4965,Mean Girls_2024_W5,Mean Girls,6,4858306,67249344,4,Paramount Pictures,2024-02-04 00:00:00+00:00,17000000,Comedy,None,None
4966,Limbo_2024_W15,Limbo,56,2862,33889,4,Music Box Films,2024-04-14 00:00:00+00:00,8000000,Adventure,Drama,Thriller
4967,Someone Like You_2024_W17,Someone Like You,26,99294,5937306,4,Fathom Events,2024-04-28 00:00:00+00:00,23000000,Comedy,Romance,None
4968,Abigail_2024_W19,Abigail,10,1577565,25230065,4,Universal Pictures,2024-05-12 00:00:00+00:00,7000000,Adventure,Fantasy,Mystery


In [23]:
df['release_month'] = df['date'].dt.month
df['release_year'] = df['date'].dt.year

df_filtered = df.drop(columns=['movie_title','unique_id', 'date', 'week_gross'])

df_filtered

,weekly_rank,current_total_gross,release_week_number,distributor,genre_1,genre_2,genre_3,release_month,release_year
0,1,5861992,1,Twentieth Century Fox,Horror,Thriller,None,6,1978
1,1,16055908,1,Paramount Pictures,Romance,Comedy,None,6,1978
2,1,540313,1,Paramount Pictures,Fantasy,Comedy,Romance,6,1978
3,1,295120,1,United Artists,Adventure,Animation,Fantasy,11,1978
4,1,12044352,1,Warner Bros.,Science Fiction,Action,Adventure,12,1978
...,...,...,...,...,...,...,...,...,...
21985,8,6331351,1,Lionsgate,Fantasy,Action,Thriller,8,2024
21986,57,765,1,Well Go USA Entertainment,Drama,Thriller,Crime,8,2024
21987,12,9083313,2,Lionsgate,Fantasy,Action,Thriller,9,2024
21988,59,4137,2,Well Go USA Entertainment,Drama,Thriller,Crime,9,2024


In [24]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
import pandas as pd

X = df_filtered.drop(columns=['current_total_gross'])  # Features
y = df_filtered['current_total_gross']  # Target

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)


In [25]:
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer

numerical_cols = ['weekly_rank', 'release_week_number', 'release_month', 'release_year']
categorical_cols = ['distributor', 'genre_1',	'genre_2',	'genre_3']

preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_cols),
        ('cat', OneHotEncoder(drop='first', handle_unknown='ignore'), categorical_cols)
    ])

X_train_processed = preprocessor.fit_transform(X_train)
X_test_processed = preprocessor.transform(X_test)


/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:242: UserWarning: Found unknown categories in columns [0, 1] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


In [26]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, r2_score
from sklearn.ensemble import RandomForestRegressor

model = RandomForestRegressor(random_state=0)
model.fit(X_train_processed, y_train)

y_pred = model.predict(X_test_processed)

mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f'Mean Absolute Error: {mae}')
print(f'R² Score: {r2}')


Mean Absolute Error: 11030710.245540854
R² Score: 0.7428796486220182


In [8]:
from sklearn.model_selection import cross_val_score

scores = cross_val_score(model, X_train_processed, y_train, cv=5, scoring='neg_mean_absolute_error')
print(f'Cross-validated MAE: {-scores.mean()}')

Cross-validated MAE: 10947482.76875386


In [31]:
import pandas as pd

# Define the new input values
new_data = {
    'weekly_rank': [1],  # Example rank
    'release_week_number': [1],  # Example release week
    'release_month': [10],  # Example release month (July)
    'release_year': [2001],  # Example release year
    'distributor': ['Warner Bros.'],  # Example distributor
    'genre_1': ['Horror'],  # Example genre 1
    'genre_2': ['Thriller'],  # Example genre 2
    'genre_3': ['']  # Example genre 3
}

# Convert to DataFrame
new_input_df = pd.DataFrame(new_data)

# Preprocess the new input data using the fitted preprocessor
new_input_processed = preprocessor.transform(new_input_df)

# Make a prediction using the trained model
predicted_gross = model.predict(new_input_processed)

# Display the predicted result
print(f'Predicted Total Gross: {predicted_gross[0]}')


Predicted Total Gross: 26453810.77


/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:242: UserWarning: Found unknown categories in columns [3] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
